In [1]:
import os
import sys
import re
import datetime
import pandas as pd
print('pandas: {}'.format(pd.__version__))
pd.set_option('display.max_rows', 500)  #horizontal scrolling
import pandas_profiling  #descriptive stats 
import numpy as np
print('numpy: {}'.format(np.__version__))

# visualizations
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
print('seaborn: {}'.format(sns.__version__))
sns.set_palette("colorblind6")
# get this from https://github.com/mwaskom/seaborn/blob/master/seaborn/palettes.py
colorblind6=["#0173B2", "#029E73", "#D55E00",
                 "#CC78BC", "#ECE133", "#56B4E9"]

pandas: 0.25.1
numpy: 1.18.1
seaborn: 0.9.0


In [2]:
# Rename product columns
def rename_products(x, roundup_match_powermax_only=False):
    # ASSAIL 30SG INSECTICIDE or Chipco
    if x.strip().upper().find("ASSAIL")>=0:
        return "ASSAIL 30SG INSECTICIDE or Chipco" 
    # BADGE X2
    elif x.strip().upper().find("BADGE")>=0:
        return "BADGE X2"
    # CHATEAU HERBICIDE SW
    elif x.strip().upper().find("CHATEAU")>=0:
        return "CHATEAU HERBICIDE SW"
    # ELEVATE 50 WDG FUNGICIDE
    elif x.strip().upper().find("ELEVATE")>=0:
        return "ELEVATE 50 WDG FUNGICIDE"
    # GOALTENDER
    elif x.strip().upper().find("GOAL")>=0:
        return "GOALTENDER"
    # INSPIRE SUPER
    elif x.strip().upper().find("INSPIRE")>=0:
        return "INSPIRE SUPER"
    # LUNA EXPERIENCE
    elif x.strip().upper().find("LUNA")>=0:
        return "LUNA EXPERIENCE"
    # MANZATE PRO-STICK FUNGICIDE
    elif x.strip().upper().find("MANZATE")>=0:
        return "MANZATE PRO-STICK FUNGICIDE"    
    # METTLE 125 ME FUNGICIDE
    elif x.strip().upper().find("METTLE")>=0:
        return "METTLE 125 ME FUNGICIDE"
    # PH-D FUNGICIDE
    elif x.strip().upper().find("PH-D")>=0:
        return "PH-D FUNGICIDE"
    # PRISTINE FUNGICIDE
    elif x.strip().upper().find("PRISTINE")>=0:
        return "PRISTINE FUNGICIDE"
    # PROLIVO 300SC FUNGICIDE
    elif x.strip().upper().find("PROLIVO")>=0:
        return "PROLIVO 300SC FUNGICIDE"
    # PROWL H2O HERBICIDE
    elif x.strip().upper().find("PROWL")>=0:
        return "PROWL H2O HERBICIDE"
    # RALLY 40 WSP
    elif x.strip().upper().find("RALLY")>=0:
        return "RALLY 40 WSP"
    # ROUNDUP POWERMAX
    elif ((x.strip().upper().find("ROUNDUP")>=0) & (roundup_match_powermax_only==False)):
        return "ROUNDUP POWERMAX"
    # ROUNDUP POWERMAX 
    elif ((x.strip().upper().find("ROUNDUP POWERMAX")>=0) & (roundup_match_powermax_only==True)):
        return "ROUNDUP POWERMAX"
    # Buccaneer
    elif (x.strip().upper().find("BUCCANEER")>=0):
        return "BUCCANEER"
    # SCALA BRAND SC FUNGICIDE
    elif x.strip().upper().find("SCALA")>=0:
        return "SCALA BRAND SC FUNGICIDE"
    # SWITCH 62.5WG
    elif x.strip().upper().find("SWITCH")>=0:
        return "SWITCH 62.5WG"
    # VIVANDO FUNGICIDE
    elif x.strip().upper().find("VIVANDO")>=0:
        return "VIVANDO FUNGICIDE"
    
    else:
        return x
    
# test it
# temp = alldf["Product Name"].apply(lambda x: rename_products(x))
# temp.value_counts().sort_index()



# convert all units to gallons
def convert_gallons(row):
    amount = row['Quantity Used']
    unit = row['Quantity Units']
    # POUND/8.33 = 1 gallons
    if unit.strip().upper().find("POUND")>=0:
        return amount/8.33 
    # PINT = 0.125 gallons
    elif unit.strip().upper().find("PINT")>=0:
        return amount * 0.125  
    # OUNCE = 0.0078125 gallons
    elif unit.strip().upper().find("OUNCE")>=0:
        return amount * 0.0078125
    # QUART = 1/4 gallons
    elif unit.strip().upper().find("QUART")>=0:
        return amount / 4    
    # Millileter = 0.000264172 gallons
    elif unit.strip().upper().find("MILLILITER")>=0:
        return amount * 0.000264172
    # Leter = 0.264172 gallons
    elif unit.strip().upper().find("LITER")>=0:
        return amount * 0.264172
    else:
        return amount
    
# test it
# temp = alldf[['Quantity Used', 'Quantity Units']].apply(lambda row: convert_gallons(row), axis=1)
# temp


# calculate pounds used
def convert_pounds(row):
    amount = row['Quantity Used']
    unit = row['Quantity Units']
    gallons = row['GallonsUsed']
    density = row['DENSITY']
    sg = row['SPEC_GRAVITY']

    # Pounds = gallons used * density * specific gravity
    if unit.strip().upper().find("POUND")>=0:
        return amount
    else:
        return gallons * density * sg


# define reporting year as Sep2017-Aug2018 = 2018; 
def calc_reporting_year(x: datetime.datetime)-> int:
    start_2018 = datetime.datetime.strptime("20170901", "%Y%m%d")
    start_2019 = datetime.datetime.strptime("20180901", "%Y%m%d")
    
    if x <= start_2019:
        return int(2018)
    else:
        return int(2019)

# test it    
# temp = alldf.loc[:, ['Application Date']]
# temp['reporting_year'] = temp['Application Date'].apply(lambda x: calc_reporting_year(x))
# temp.head()

## Read Megan's Product to Active Ingredient table

In [3]:
# read data
dataFile = '/Users/christybergman/Documents/personal/pesticideVolunteerWork/Megan_VineyardPesticides-MostUsed 2.xlsx'
lookup = pd.read_excel(dataFile, header=1)
print(lookup.shape)

(39, 6)


In [4]:
lookup.columns.tolist()

['Active Ingredient',
 'Product Name(s)',
 'Product Type',
 'PANNA Search www.PesticideInfo.org',
 'Beyond Pesticides Gateway Search\nhttps://www.beyondpesticides.org/resources/pesticide-gateway',
 'Other health effects/references']

In [6]:
# Clean up product names
lookup.columns = ['Active Ingredient',
 'Product Name(s)',
 'Product Type',
 'PANNA Search www.PesticideInfo.org',
 'Beyond Pesticides Gateway https://www.beyondpesticides.org/resources/pesticide-gateway',
 'Other health effects/references']
# drop columns without any product names - drops 1 row
lookup = lookup.loc[(lookup['Product Name(s)'].notna()), :]
# strip whitespace and convert all capitals
lookup.loc[:, 'join_product'] = lookup['Product Name(s)'].apply(lambda x: x.strip().upper())

# Save to csv
lookup.to_csv('Megan_file_clean.csv', sep='\t')

print(lookup.shape)
lookup.tail()

(37, 7)


/Users/christybergman/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/christybergman/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Active Ingredient,Product Name(s),Product Type,PANNA Search www.PesticideInfo.org,Beyond Pesticides Gateway https://www.beyondpesticides.org/resources/pesticide-gateway,Other health effects/references,join_product
32,KRESOXIM-METHYL,SOVRAN FUNGICIDE,Fungicide,"PAN Bad Actor\nCarcinogen, potential water con...","Likely carcinogen, toxic to fish/aquatic organ...",NaN,SOVRAN FUNGICIDE
33,FLUDIOXONIL,SWITCH 62.5WG,Fungicide,Potential Water Contaminant,"Kidney/liver damage, sensitizer/irritant, toxi...",NaN,SWITCH 62.5WG
34,CYFLUFENAMID,TORINO,Fungicide,PAN Bad Actor\nCarcinogen,NaN,"Likely to be Carcinogenic to humans, toxic to ...",TORINO
35,TRIFLUMIZOLE,VITICURE,Fungicide,Water contaminant,"Reproductive effects, kidney/liver damage, sen...",NaN,VITICURE
36,METRAFENONE\t,VIVANDO FUNGICIDE,Fungicide,NaN,NaN,NaN,VIVANDO FUNGICIDE


## Read CDPR Product Tables

Convert pounds = Gallons * Density * Specific Gravity of the product

In [7]:
# Get fixed widths from https://www.cdpr.ca.gov/docs/label/table_formats.pdf
fwidths = [7, 7, 7, 5, 2, 7, 1, 100, 24, 1, 1, 1, 7, 2, 11, 11, 1, 1
           , 11, 1, 11, 11, 1, 1, 1, 1, 1, 8, 1, 1]
colnames = ["PRODNO", "MFG_FIRMNO", "REG_FIRMNO", "LABEL_SEQ_NO", "REVISION_NO", "FUT_FIRMNO"
         , "PRODSTAT_IND", "PRODUCT_NAME", "SHOW_REGNO", "AER_GRND_IND", "AGRICCOM_SW", "CONFID_SW"
         , "DENSITY", "FORMULA_CD", "FULL_EXP_DT", "FULL_ISS_DT", "FUMIGANT_SW", "GEN_PEST_IND"
         , "LASTUP_DT", "MFG_REF_SW", "PROD_INAC_DT", "REG_DT", "REG_TYPE_IND", "RODENT_SW"
         , "SIGNLWRD_IND", "SOILAPPL_SW", "SPECGRAV_SW", "SPEC_GRAVITY", "CONDREG_SW", "VAR2_SW"]

# Read the products from zip file https://apps.cdpr.ca.gov/docs/label/labelque.cfm
products = pd.DataFrame()
path = "/Users/christybergman/Documents/personal/pesticideVolunteerWork/productDatabase/product.txt"
products = pd.read_fwf(path, widths = fwidths,
               names = colnames)
print(products.shape)

# Check some rows
products.loc[(products['PRODNO'].isin([57011, 65479]))
             , ["PRODNO", "PRODUCT_NAME", "DENSITY", "SPECGRAV_SW", "SPEC_GRAVITY"]].head()

(65214, 30)


,PRODNO,PRODUCT_NAME,DENSITY,SPECGRAV_SW,SPEC_GRAVITY
22252,57011,ROUNDUP POWERMAX HERBICIDE,11.331,A,1.3599
23522,65479,ROUNDUP READY-TO-USE WEED & GRASS KILLER PLUS ...,8.980,A,1.0778


In [8]:
# Rename products columns
products.loc[:, "join_product"] = products["PRODUCT_NAME"]\
        .apply(lambda x: rename_products(x, roundup_match_powermax_only=True))
# Check some rows
products.loc[(products['PRODNO'].isin([57011, 65479]))
             , ["PRODNO", "PRODUCT_NAME", "DENSITY", "SPECGRAV_SW", "SPEC_GRAVITY", "join_product"]].head()

,PRODNO,PRODUCT_NAME,DENSITY,SPECGRAV_SW,SPEC_GRAVITY,join_product
22252,57011,ROUNDUP POWERMAX HERBICIDE,11.331,A,1.3599,ROUNDUP POWERMAX
23522,65479,ROUNDUP READY-TO-USE WEED & GRASS KILLER PLUS ...,8.980,A,1.0778,ROUNDUP READY-TO-USE WEED & GRASS KILLER PLUS ...


## Read all the newest FOIA'd Permit Use Reports

In [9]:
# For every available file, read the file, append to master dataframe:
alldf = pd.DataFrame()

path = "/Users/christybergman/Documents/personal/pesticideVolunteerWork/wellWaterGrant/PURs/"
files = os.listdir(path)
# keyword = 'your_keyword'
for name in files:
    filename = os.path.join(path, name)
#     print(filename)
    df = pd.read_excel(filename, sheet_name="PermitUseReport")
#     print(f"shape new file = {df.shape}")
    alldf = pd.concat([alldf, df], axis=0)
    
print(f"shape all data = {alldf.shape}")  
alldf.head(2)

shape all data = (1801, 16)


,Permit #,Permitee,Site ID,Site Location,Site Name,Application Date,Application Time,Commodity,Planted Amount,Planted Units,EPA Reg No,Product Name,Quantity Used,Quantity Units,Treated Amount,Treated Units
0,494728,APPLE BOTTOM FARM,1,(130-270-011),8490 OCCIDENTAL RD,2018-02-25,8:30 AM,APPLE,2.5,ACRES,80289-12-AA,BADGE X2,7.5,Pounds,2.5,ACRES
1,494728,APPLE BOTTOM FARM,1,(130-270-011),8490 OCCIDENTAL RD,2018-02-25,8:30 AM,APPLE,2.5,ACRES,34704-1028-AA,LEAF LIFE GAVICIDE GREEN 415,1.5,Gallon,2.5,ACRES


In [65]:
# drop null rows - drops 2 rows out of of 1801 rows
print(alldf.shape)
alldf = alldf.loc[(alldf['Quantity Used'].notna()), :]
print(alldf.shape)

# drop duplicates
print(alldf.shape)
alldf.drop_duplicates(inplace=True)
print(alldf.shape)

# add year column
alldf.loc[:, 'year'] = alldf['Application Date'].dt.year  #.astype('int64')
print(f"min date = {alldf['Application Date'].min()}")
print(f"max date = {alldf['Application Date'].max()}")
print(alldf.year.value_counts(dropna=False))
      
# add reporting year column
alldf.loc[:, 'reportingYear'] = alldf['Application Date'].apply(lambda x: calc_reporting_year(x))
print(alldf.reportingYear.value_counts(dropna=False))
      
# split site into site_name (BATEMAN) and site_parcel (061-030-011)
      
# strip whitespace and convert all capitals
alldf.loc[:, 'Quantity Units'] = alldf['Quantity Units'].apply(lambda x: x.strip().upper())
# convert units
alldf.loc[:, "GallonsUsed"] = alldf[['Quantity Used', 'Quantity Units']].apply(lambda row: convert_gallons(row), axis=1)
print(alldf["Quantity Units"].value_counts(dropna=False))
print(alldf.loc[:, ["GallonsUsed", 'Quantity Used', 'Quantity Units']].head())  
      
# Rename product columns
alldf.loc[:, "join_product"] = alldf["Product Name"].apply(lambda x: rename_products(x))
alldf["join_product"].value_counts().sort_index()
      
# Add City, State, Zip
alldf['zipcode'] = 95444
alldf['city'] = "Graton"
alldf["state"] = "California"

(1799, 20)
(1799, 20)
(1799, 20)
(1799, 20)
min date = 2017-01-11 00:00:00
max date = 2019-08-23 00:00:00
2018    903
2019    880
2017     16
Name: year, dtype: int64
2019    927
2018    872
Name: reportingYear, dtype: int64
POUNDS         499
PINT           479
GALLON         447
OUNCE          351
QUART           20
MILLILITERS      2
LITERS           1
Name: Quantity Units, dtype: int64
   GallonsUsed  Quantity Used Quantity Units
0      0.90036            7.5         POUNDS
1      1.50000            1.5         GALLON
2      1.20048           10.0         POUNDS
3      0.03125            4.0          OUNCE
4      0.60024            5.0         POUNDS


In [68]:
alldf.head(2)

,Permit #,Permitee,Site ID,Site Location,Site Name,Application Date,Application Time,Commodity,Planted Amount,Planted Units,...,Quantity Units,Treated Amount,Treated Units,year,reportingYear,GallonsUsed,join_product,zipcode,city,state
0,494728,APPLE BOTTOM FARM,1,(130-270-011),8490 OCCIDENTAL RD,2018-02-25,8:30 AM,APPLE,2.5,ACRES,...,POUNDS,2.5,ACRES,2018,2018,0.90036,BADGE X2,95444,Graton,California
1,494728,APPLE BOTTOM FARM,1,(130-270-011),8490 OCCIDENTAL RD,2018-02-25,8:30 AM,APPLE,2.5,ACRES,...,GALLON,2.5,ACRES,2018,2018,1.50000,LEAF LIFE GAVICIDE GREEN 415,95444,Graton,California


In [11]:
# How many owners?
print(f"Number of owners: {alldf.Permitee.nunique()}")
print(f"Number of vineyards: {alldf['Site Location'].nunique()}")

Number of owners: 16
Number of vineyards: 35


In [73]:
# Download distinct addresses
temp = alldf.loc[:, ["Site Name", 'city', 'state', 'zipcode']].groupby("Site Name").first()
temp.reset_index(inplace=True, drop=False)

temp.to_csv("DistinctAddresses.csv"
                 ,index=False,sep="\t"
                 , header=True
                 , encoding='utf-8'
                 , quoting=csv.QUOTE_NONNUMERIC)

# Translate using https://dash.geocod.io/import

In [82]:
# Upload the geo-codeded addresses and append new columns  
path = "/Users/christybergman/Documents/python_code/pythonHomeworks/pesticides/DistinctAddresses_geocodio.xlsm"
name = "DistinctAddresses_geocodio.xlsm"
filename = os.path.join(path, name)
print(filename)

geo = pd.read_excel(path, header=0)
print(geo.shape)
print(geo.head(2))

# merge the lat, lon
geo_latlon = pd.concat([temp, geo], axis=1)
geo_latlon.tail()

/Users/christybergman/Documents/python_code/pythonHomeworks/pesticides/DistinctAddresses_geocodio.xlsm/DistinctAddresses_geocodio.xlsm
(37, 12)


,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,City,State,County,Zip,Country,Source
0,38.422620,-122.877608,0.9,rooftop,10210.0,Mill Station Rd,Sebastopol,CA,Sonoma County,95472,US,Sonoma County
1,38.424667,-122.877818,0.9,rooftop,10404.0,Mill Station Rd,Sebastopol,CA,Sonoma County,95472,US,Sonoma County


In [95]:
#concat lat, long
merge_geo = alldf.copy()
print(merge_geo.shape)
merge_geo = merge_geo.merge(geo_latlon, on="Site Name"
                       , how="left"
#                       , sort=True
                      , copy=False)
#                       , indicator=True)
# merge_geo.drop_duplicates(inplace=True)
print(merge_geo.shape)
merge_geo.head()

(1799, 23)
(1799, 38)


,Permit #,Permitee,Site ID,Site Location,Site Name,Application Date,Application Time,Commodity,Planted Amount,Planted Units,...,Accuracy Score,Accuracy Type,Number,Street,City,State,County,Zip,Country,Source
0,494728,APPLE BOTTOM FARM,1,(130-270-011),8490 OCCIDENTAL RD,2018-02-25,8:30 AM,APPLE,2.5,ACRES,...,0.9,rooftop,8490.0,Occidental Rd,Sebastopol,CA,Sonoma County,95472,US,Sonoma County
1,494728,APPLE BOTTOM FARM,1,(130-270-011),8490 OCCIDENTAL RD,2018-02-25,8:30 AM,APPLE,2.5,ACRES,...,0.9,rooftop,8490.0,Occidental Rd,Sebastopol,CA,Sonoma County,95472,US,Sonoma County
2,494728,APPLE BOTTOM FARM,1,(130-270-011),8490 OCCIDENTAL RD,2018-04-04,9:00 AM,APPLE,2.5,ACRES,...,0.9,rooftop,8490.0,Occidental Rd,Sebastopol,CA,Sonoma County,95472,US,Sonoma County
3,494728,APPLE BOTTOM FARM,1,(130-270-011),8490 OCCIDENTAL RD,2018-04-04,9:00 AM,APPLE,2.5,ACRES,...,0.9,rooftop,8490.0,Occidental Rd,Sebastopol,CA,Sonoma County,95472,US,Sonoma County
4,494728,APPLE BOTTOM FARM,1,(130-270-011),8490 OCCIDENTAL RD,2018-04-17,10:00 AM,APPLE,2.5,ACRES,...,0.9,rooftop,8490.0,Occidental Rd,Sebastopol,CA,Sonoma County,95472,US,Sonoma County


In [96]:
#join with Megan's table
merge_df = merge_geo.copy()
print(merge_df.shape)
merge_df = merge_df.merge(lookup, on="join_product"
                       , how="left"
                      , sort=True
                      , copy=False
                      , indicator=True)
# print(merge_df.shape)
merge_df.drop_duplicates(inplace=True)
print(merge_df.shape)

# Note: lookup has more than 1 row per product name, so merge result could have more rows than original
# check you did the right thing
print(f"types of joins: {merge_df['_merge'].value_counts(dropna=False)}")
print(f"Number of new rows in join: {1921 - 1799}")
orig_rows_multiple_chemicals = alldf.loc[((alldf['Product Name']=="INSPIRE SUPER")
           | (alldf['Product Name']=="LUNA EXPERIENCE")
           | (alldf['Product Name']=="PRISTINE FUNGICIDE")), :].shape
print(f"Number of original rows with multiple chemicals per product: {orig_rows_multiple_chemicals}")
print("Success, number of rows match up!")
# Success, rows match up!   
      
merge_df.drop("_merge", axis=1, inplace=True)
# print(merge_df.shape)
merge_df.loc[(merge_df['Active Ingredient'].notna()), :].head()

(1799, 38)
(1921, 45)
types of joins: left_only     1205
both           716
right_only       0
Name: _merge, dtype: int64
Number of new rows in join: 122
Number of original rows with multiple chemicals per product: (104, 23)
Success, number of rows match up!


,Permit #,Permitee,Site ID,Site Location,Site Name,Application Date,Application Time,Commodity,Planted Amount,Planted Units,...,County,Zip,Country,Source,Active Ingredient,Product Name(s),Product Type,PANNA Search www.PesticideInfo.org,Beyond Pesticides Gateway https://www.beyondpesticides.org/resources/pesticide-gateway,Other health effects/references
25,490225,DUTTON RANCH CORP,84,(GOFF) (130-270-043),8760 OCCIDENTAL RD,2018-02-12,11:14 AM,"GRAPE, WINE",33.00,ACRES,...,Sonoma County,95472,US,Sonoma County,INDAZIFLAM,ALION HERBICIDE,Herbicide,NaN,"Endocrine disruptor, neurotoxicity, kidney/liv...",Suspected neurotoxin\nhttps://www3.epa.gov/pes...
26,490225,DUTTON RANCH CORP,47 B,(CEMETERY) (130-320-005),3003 SULLIVAN RD,2018-02-12,11:09 AM,"GRAPE, WINE",16.00,ACRES,...,Sonoma County,95472,US,Sonoma County,INDAZIFLAM,ALION HERBICIDE,Herbicide,NaN,"Endocrine disruptor, neurotoxicity, kidney/liv...",Suspected neurotoxin\nhttps://www3.epa.gov/pes...
27,493138,VINEPRO VINEYARD MANAGEMENT,63,(Edwards-Coopersmith) (130-180-088),2959 HIGHWAY 116,2019-01-23,1:00 AM,"GRAPE, WINE",6.52,ACRES,...,Sonoma County,95472,US,Sonoma County,INDAZIFLAM,ALION HERBICIDE,Herbicide,NaN,"Endocrine disruptor, neurotoxicity, kidney/liv...",Suspected neurotoxin\nhttps://www3.epa.gov/pes...
28,490225,DUTTON RANCH CORP,10,(WIDDOES) (130-090-016),3515 SULLIVAN RD,2018-02-12,11:09 AM,"GRAPE, WINE",30.00,ACRES,...,Sonoma County,95472,US,Sonoma County,INDAZIFLAM,ALION HERBICIDE,Herbicide,NaN,"Endocrine disruptor, neurotoxicity, kidney/liv...",Suspected neurotoxin\nhttps://www3.epa.gov/pes...
29,492947,KIRK LOKKA,4,(130-261-037),2325 GRAVENSTEIN HWY NORTH,2017-12-05,3:00 PM,"GRAPE, WINE",6.00,ACRES,...,Sonoma County,95472,US,TIGER/Line¬Æ dataset from the US Census Bureau,INDAZIFLAM,ALION HERBICIDE,Herbicide,NaN,"Endocrine disruptor, neurotoxicity, kidney/liv...",Suspected neurotoxin\nhttps://www3.epa.gov/pes...


In [97]:
#join with cdpr Product Labels table
merge_df2 = merge_df.copy()
products2 = products.copy()
products2 = products2[["DENSITY", "SPEC_GRAVITY", "join_product"]].groupby("join_product").first()
products2.head()
print(merge_df2.shape)
merge_df2 = merge_df2.merge(products2, on="join_product"
                      , how="left"
                      , sort=True
                      , copy=False
                      , indicator=True)

# If no density or specific gravity, then assume 8.332 density and s.g.=1
merge_df2.loc[(merge_df2["_merge"]=="left_only"), 'DENSITY'] = 8.332
merge_df2.loc[(merge_df2["_merge"]=="left_only"), 'SPEC_GRAVITY'] = 1.0

# print(merge_df.shape)
print(merge_df2.shape)
merge_df2.drop("_merge", inplace=True, axis=1)
merge_df2.drop_duplicates(inplace=True)
print(merge_df2.shape)

(1921, 44)
(1921, 47)
(1921, 46)


In [98]:
# Check you did the right thing
print(merge_df2.loc[((merge_df2['join_product']=="ROUNDUP POWERMAX")
           | (merge_df2['join_product']=="BUCCANEER")), :]['GallonsUsed'].sum())

print(alldf.loc[((alldf['join_product']=="ROUNDUP POWERMAX")
           | (alldf['join_product']=="BUCCANEER")), :]['GallonsUsed'].sum())

90.48862512500001
90.48862512499997


In [99]:
merge_df2.tail(2)

,Permit #,Permitee,Site ID,Site Location,Site Name,Application Date,Application Time,Commodity,Planted Amount,Planted Units,...,Country,Source,Active Ingredient,Product Name(s),Product Type,PANNA Search www.PesticideInfo.org,Beyond Pesticides Gateway https://www.beyondpesticides.org/resources/pesticide-gateway,Other health effects/references,DENSITY,SPEC_GRAVITY
1919,494711,BIG PIG VINEYARD,1,(130-090-011),3115 SULLIVAN ROAD,2019-07-07,8:00 AM,"GRAPE, WINE",4.0,ACRES,...,US,Sonoma County,NaN,NaN,NaN,NaN,NaN,NaN,8.49,1.019
1920,494711,BIG PIG VINEYARD,1,(130-090-011),3115 SULLIVAN ROAD,2019-07-27,5:30 AM,"GRAPE, WINE",4.0,ACRES,...,US,Sonoma County,NaN,NaN,NaN,NaN,NaN,NaN,8.49,1.019


In [100]:
merge_df2['Quantity Units'].value_counts(dropna=False)

PINT           527
POUNDS         501
GALLON         453
OUNCE          417
QUART           20
MILLILITERS      2
LITERS           1
Name: Quantity Units, dtype: int64

In [101]:
# Calculate Pounds Applied = pounds or gallons * density * specific gravity
merge_df2.loc[:, "PoundsApplied"] = merge_df2[["GallonsUsed", 'Quantity Used'
                                           , 'Quantity Units', "DENSITY", "SPEC_GRAVITY"]]\
        .apply(lambda row: convert_pounds(row), axis=1)
print(merge_df2.loc[:, ["GallonsUsed", 'Quantity Used', 'Quantity Units', "PoundsApplied"]].tail()) 

      GallonsUsed  Quantity Used Quantity Units  PoundsApplied
1916     0.175781          22.50          OUNCE       1.464609
1917     0.156250          20.00          OUNCE       1.301875
1918     0.194609          24.91          OUNCE       1.621485
1919     0.093750          12.00          OUNCE       0.811060
1920     0.093750          12.00          OUNCE       0.811060


## Read historical PUR reports

In [19]:
# read data
path = "/Users/christybergman/Documents/personal/pesticideVolunteerWork/wellWaterGrant/historicalPURs/7530917630382_200224101339/"
name = "7530917630382_200224101339.txt"
filename = os.path.join(path, name)
print(filename)

# dataFile = "/Users/christybergman/Documents/personal/pesticideVolunteerWork/wellWaterGrant/historicalPURs/7530917630382_200224101339/7530917630382_200224101339⁩.txt"
df2017 = pd.read_csv(filename, header=0, sep="\t")
print(df2017.shape)
df2017.head(2)

/Users/christybergman/Documents/personal/pesticideVolunteerWork/wellWaterGrant/historicalPURs/7530917630382_200224101339/7530917630382_200224101339.txt
(2407, 13)


,ADJUVANT,YEAR,DATE,COUNTY_NAME,COMTRS,SITE_NAME,PRODUCT_NAME,POUNDS_PRODUCT_APPLIED,CHEMICAL_NAME,POUNDS_CHEMICAL_APPLIED,AMOUNT_TREATED,UNIT_TREATED,AERIAL_GROUND_INDICATOR
0,NO,2017,30-AUG-17,SONOMA,49M07N09W27,ALFALFA (FORAGE - FODDER) (ALFALFA HAY),MULTI-CHLOR,2.5407,SODIUM HYPOCHLORITE,0.317588,55.0,?,O
1,NO,2017,12-JAN-17,SONOMA,49M07N09W27,ALFALFA (FORAGE - FODDER) (ALFALFA HAY),MULTI-CHLOR,5.0813,SODIUM HYPOCHLORITE,0.635162,220.0,?,O
2,NO,2017,16-MAR-17,SONOMA,49M07N09W27,ALFALFA (FORAGE - FODDER) (ALFALFA HAY),MULTI-CHLOR,10.1626,SODIUM HYPOCHLORITE,1.270325,660.0,?,O
3,NO,2017,20-MAR-17,SONOMA,49M07N09W27,ALFALFA (FORAGE - FODDER) (ALFALFA HAY),MULTI-CHLOR,10.1626,SODIUM HYPOCHLORITE,1.270325,385.0,?,O
4,NO,2017,15-MAY-17,SONOMA,49M07N09W27,ALFALFA (FORAGE - FODDER) (ALFALFA HAY),MULTI-CHLOR,10.1626,SODIUM HYPOCHLORITE,1.270325,880.0,?,O


In [49]:
# Add expected columns for joining
df2017['reportingYear'] = df2017.YEAR
df2017['PoundsApplied'] = df2017['POUNDS_CHEMICAL_APPLIED']
df2017["Application Date"] = df2017["DATE"]
df2017["Planted Amount"] = df2017["AMOUNT_TREATED"]
df2017.loc[:, "join_product"] = df2017["PRODUCT_NAME"].apply(lambda x: rename_products(x))
df2017.head(2)

,ADJUVANT,YEAR,DATE,COUNTY_NAME,COMTRS,SITE_NAME,PRODUCT_NAME,POUNDS_PRODUCT_APPLIED,CHEMICAL_NAME,POUNDS_CHEMICAL_APPLIED,AMOUNT_TREATED,UNIT_TREATED,AERIAL_GROUND_INDICATOR,reportingYear,PoundsApplied,join_product,Application Date,Planted Amount
0,NO,2017,30-AUG-17,SONOMA,49M07N09W27,ALFALFA (FORAGE - FODDER) (ALFALFA HAY),MULTI-CHLOR,2.5407,SODIUM HYPOCHLORITE,0.317588,55.0,?,O,2017,0.317588,MULTI-CHLOR,30-AUG-17,55.0
1,NO,2017,12-JAN-17,SONOMA,49M07N09W27,ALFALFA (FORAGE - FODDER) (ALFALFA HAY),MULTI-CHLOR,5.0813,SODIUM HYPOCHLORITE,0.635162,220.0,?,O,2017,0.635162,MULTI-CHLOR,12-JAN-17,220.0


In [ ]:
#join with Megan's table
merge_df3 = df2017.copy()
print(merge_df3.shape)
merge_df3 = merge_df3.merge(lookup, on="join_product"
                       , how="left"
                      , sort=True
                      , copy=False
                      , indicator=True)
# print(merge_df.shape)
merge_df3.drop_duplicates(inplace=True)
print(merge_df3.shape)

# Note: lookup has more than 1 row per product name, so merge result could have more rows than original   
# merge_df3.drop("_merge", axis=1, inplace=True)
# print(merge_df.shape)
merge_df3.loc[(merge_df3['Active Ingredient'].notna()), :].shape

## Create csv for Tableau visualizations

In [64]:
# Save as csv
import csv

merge_df3.to_csv("Historical2017.csv"
                 ,index=False,sep="\t"
                 #, header=True
                 , encoding='utf-8'
                 , quoting=csv.QUOTE_NONNUMERIC)

In [63]:
# Check you did the right thing
print(f"Distinct products: {merge_df3['PRODUCT_NAME'].nunique()}")
print(f"Total pounds used: {merge_df3.groupby('reportingYear')['PoundsApplied'].sum()}")

print(merge_df3.loc[((merge_df3['join_product']=="ROUNDUP POWERMAX")
           | (merge_df3['join_product']=="BUCCANEER")), :]['PoundsApplied'].sum())

print(merge_df3.loc[((merge_df3['join_product']=="ROUNDUP POWERMAX")
           | (merge_df3['join_product']=="BUCCANEER")), :]['PoundsApplied'].sum())

Distinct products: 125
Total pounds used: reportingYear
2017    35795.374644
Name: PoundsApplied, dtype: float64
547.699098
547.699098


In [102]:
# Save as csv
# merge_df2.to_csv('PUR_2018_2019.csv', sep=';', header=True)
import csv

merge_df2.to_csv("PUR_2018_2019.csv"
                 ,index=False,sep="\t"
                 , header=True
                 , encoding='utf-8'
                 , quoting=csv.QUOTE_NONNUMERIC)

In [103]:
# Check you did the right thing
print(f"Distinct owners: {merge_df2['Permitee'].nunique()}")
print(f"Distinct products: {merge_df2['Product Name'].nunique()}")
print(f"Total pounds used: {merge_df2.groupby('reportingYear')['PoundsApplied'].sum()}")

print(merge_df2.loc[((merge_df2['join_product']=="ROUNDUP POWERMAX")
           | (merge_df2['join_product']=="BUCCANEER")), :]['PoundsApplied'].sum())

print(merge_df2.loc[((merge_df2['join_product']=="ROUNDUP POWERMAX")
           | (merge_df2['join_product']=="BUCCANEER")), :]['PoundsApplied'].sum())

Distinct owners: 16
Distinct products: 139
Total pounds used: reportingYear
2018    60665.346089
2019    47419.205233
Name: PoundsApplied, dtype: float64
1121.4060043501408
1121.4060043501408
